In [19]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium import webdriver
import time
import pandas as pd
import eventlet
import random

In [44]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=r"D:\Chrome Downloads\chromedriver_win32\chromedriver.exe", options=options)
    driver.set_window_size(1120, 1000)

#     url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword + '&locT=&locId=0&locKeyword=&srs=RECENT_SEARCHES'
    driver.get(url)
    jobs = []

#     eventlet.monkey_patch()
#     with eventlet.Timeout(120,False):
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.
        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(random.random())

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException or StaleElementReferenceException:
            pass

        time.sleep(random.random())

        try:
#             driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
            driver.find_element_by_css_selector("svg[class='SVGInline-svg modal_closeIcon-svg']").click()
        except NoSuchElementException:
            pass


        #Going through each job in this page
#         try:
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
#         except ElementNotInteractableException:
#             pass

        for job_button in job_buttons:  
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            time.sleep(random.random())
            job_button.click()  #You might 
#             time.sleep(5)
            # scroll to the job button
            driver.execute_script("arguments[0].scrollIntoView();", job_button)
        
            collected_successfully = False

            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(random.random())

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."

            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1


            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1


            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs
            
            # scroll 10 pixels
            # js = "var q=document.documentElement.scrollTop=100"
            # driver.execute_script(js)
     

        #Clicking on the "next page" button
#         try:
#             driver.find_element_by_xpath('//*[@id="FooterPageNav"]/div/ul/li[4]').click()
# #             driver.find_element_by_xpath('.//li[@class="next"]//a').click()
#         except NoSuchElementException:
#             print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
#             break
            
        next_page_btn = driver.find_element_by_xpath('//*[@id="FooterPageNav"]/div/ul/li[last()]')
        if driver.find_element_by_xpath('//*[@id="FooterPageNav"]/div/ul/li[last()]/a/span').get_attribute("class") == "disabled":
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
        else:
            next_page_btn.click()

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [46]:
#This line will open a new chrome window and start the scraping.
# df = get_jobs("data scientist", 5, False)
df = get_jobs("Abbvie", 5000, False)
df

Progress: 0/5000
Progress: 1/5000
Progress: 2/5000
Progress: 3/5000
Progress: 4/5000
Progress: 5/5000
Progress: 6/5000
Progress: 7/5000
Progress: 8/5000
Progress: 9/5000
Progress: 10/5000
Progress: 11/5000
Progress: 12/5000
Progress: 13/5000
Progress: 14/5000
Progress: 15/5000
Progress: 16/5000
Progress: 17/5000
Progress: 18/5000
Progress: 19/5000
Progress: 20/5000
Progress: 21/5000
Progress: 22/5000
Progress: 23/5000
Progress: 24/5000
Progress: 25/5000
Progress: 26/5000
Progress: 27/5000
Progress: 28/5000
Progress: 29/5000
Progress: 30/5000
Progress: 31/5000
Progress: 32/5000
Progress: 33/5000
Progress: 34/5000
Progress: 35/5000
Progress: 36/5000
Progress: 37/5000
Progress: 38/5000
Progress: 39/5000
Progress: 40/5000
Progress: 41/5000
Progress: 42/5000
Progress: 43/5000
Progress: 44/5000
Progress: 45/5000
Progress: 46/5000
Progress: 47/5000
Progress: 48/5000
Progress: 49/5000
Progress: 50/5000
Progress: 51/5000
Progress: 52/5000
Progress: 53/5000
Progress: 54/5000
Progress: 55/5000
Pr

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=83.0.4103.61)


In [32]:
df.to_csv('Abbvie.csv', index=False)

In [43]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Research Associate,-1,Agilent inspires and supports discoveries that...,4.0,Agilent Technologies\n4.0,"Carpinteria, CA","Santa Clara, CA",10000+ employees,1999,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),"Thermo Fisher Scientific, Illumina, Danaher"
1,Complaints Investigator,-1,Agilent inspires and supports discoveries that...,4.0,Agilent Technologies\n4.0,"Cedar Creek, TX","Santa Clara, CA",10000+ employees,1999,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),"Thermo Fisher Scientific, Illumina, Danaher"
2,Complaint Handling Lead,-1,Agilent inspires and supports discoveries that...,4.0,Agilent Technologies\n4.0,"Cedar Creek, TX","Santa Clara, CA",10000+ employees,1999,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),"Thermo Fisher Scientific, Illumina, Danaher"
3,Research Assistant,-1,Agilent inspires and supports discoveries that...,4.0,Agilent Technologies\n4.0,"Cedar Creek, TX","Santa Clara, CA",10000+ employees,1999,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),"Thermo Fisher Scientific, Illumina, Danaher"
4,MDR Vigilance Specialist Manager,-1,Agilent inspires and supports discoveries that...,4.0,Agilent Technologies\n4.0,"Cedar Creek, TX","Santa Clara, CA",10000+ employees,1999,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),"Thermo Fisher Scientific, Illumina, Danaher"
5,Histologist,-1,Agilent inspires and supports discoveries that...,4.0,Agilent Technologies\n4.0,"Carpinteria, CA","Santa Clara, CA",10000+ employees,1999,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),"Thermo Fisher Scientific, Illumina, Danaher"
6,"Research Associate (Intermediate), Companion D...",-1,Agilent inspires and supports discoveries that...,4.0,Agilent Technologies\n4.0,"Carpinteria, CA","Santa Clara, CA",10000+ employees,1999,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),"Thermo Fisher Scientific, Illumina, Danaher"
7,Sales - Atomic Spectroscopy/Inorganic Instrument,-1,"Agilent is seeking an enthusiastic, results dr...",4.0,Agilent Technologies\n4.0,"Houston, TX","Santa Clara, CA",10000+ employees,1999,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),"Thermo Fisher Scientific, Illumina, Danaher"
8,Operational Excellence Manager,-1,Agilent inspires and supports discoveries that...,4.0,Agilent Technologies\n4.0,"Houston, TX","Santa Clara, CA",10000+ employees,1999,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),"Thermo Fisher Scientific, Illumina, Danaher"
9,"Project Manager, Companion Diagnostics",-1,Be a part of the future of Precision Medicine!...,4.0,Agilent Technologies\n4.0,"Carpinteria, CA","Santa Clara, CA",10000+ employees,1999,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),"Thermo Fisher Scientific, Illumina, Danaher"
